# Fantasy Football Schedule Generator
In this tutorial I will show you how to:
1. Get the teams from yahoo
2. Generate the schedule with various options
3. Validate the schedule
4. Upload the schedule (manual for now but might make a vid on automating it)


## Get the Schedule

In [79]:
#Copy and paste the text for your league. 
team_str="""
logo	Ben’s Koo Young	Tom	Jul 6	Created League
logo	The Half Chubbs	Joshua	Jul 6	Joined via renew
logo	Bend It Like Beckham	Mariano	Jul 6	Joined via renew
logo	Football Team	Eric	Jul 6	Joined via renew
logo	Keep Dragon Shoulder	Greg	Jul 6	Joined via renew
logo	#3	Matt	Jul 6	Joined via renew
logo	Stafford Infection	Eric	Jul 6	Joined via renew
logo	Jake's Liver 2	Jake	Jul 6	Joined via renew
logo	AutodraftKings	Nick	Jul 6	Joined via renew
logo	White Sox Hat	Lucas	Jul 6	Joined via renew
logo	7th Floor Crew	Edwin	Jul 6	Joined via renew
logo	Real Edwards	Jason	Jul 6	Joined via renew
logo	Just Win Baby	Matteo	Jul 6	Joined via renew
logo	Bishop Sycamore	Chris	Jul 6	Joined via renew
logo	OnlyFants	Ankit	Jul 6	Joined via renew
logo	Mack Top Greg Bottom	Nick	Jul 6	Joined via renew
"""

In [93]:
all_teams=[x.split("	")[1] for x in team_str.split("\n")[1:-1]]
all_teams,len(all_teams)

(['Ben’s Koo Young',
  'The Half Chubbs',
  'Bend It Like Beckham',
  'Football Team',
  'Keep Dragon Shoulder',
  '#3',
  'Stafford Infection',
  "Jake's Liver 2",
  'AutodraftKings',
  'White Sox Hat',
  '7th Floor Crew',
  'Real Edwards',
  'Just Win Baby',
  'Bishop Sycamore',
  'OnlyFants',
  'Mack Top Greg Bottom'],
 16)

In [81]:
import random
random.shuffle(all_teams)#shuffles in place. Makes everything "random" -- technically pseudorandom but still should be good enough
all_teams

['OnlyFants',
 'Mack Top Greg Bottom',
 'Bend It Like Beckham',
 'The Half Chubbs',
 '#3',
 "Jake's Liver 2",
 'Stafford Infection',
 'White Sox Hat',
 'Keep Dragon Shoulder',
 'Real Edwards',
 'AutodraftKings',
 '7th Floor Crew',
 'Football Team',
 'Ben’s Koo Young',
 'Bishop Sycamore',
 'Just Win Baby']

To preserve the randomness you can set a "seed"  https://docs.python.org/3/library/random.html#random.seed but we just save the output later instead.

Since we randomized earlier we can use deterministic methods like %, the modulo operator, to simplify our code.

In [82]:
from collections import defaultdict#makes list append simpler

divisions=defaultdict(list)
for i,team in enumerate(all_teams):
    divisions[i//4].append(team)#every 4th team from the random list is assigned to the same division
divisions

defaultdict(list,
            {0: ['OnlyFants',
              'Mack Top Greg Bottom',
              'Bend It Like Beckham',
              'The Half Chubbs'],
             1: ['#3',
              "Jake's Liver 2",
              'Stafford Infection',
              'White Sox Hat'],
             2: ['Keep Dragon Shoulder',
              'Real Edwards',
              'AutodraftKings',
              '7th Floor Crew'],
             3: ['Football Team',
              'Ben’s Koo Young',
              'Bishop Sycamore',
              'Just Win Baby']})

In [83]:

conferences=defaultdict(list)
conference_nums=[]
for division, teams in divisions.items():
    division_num=len(teams)//2#2 conferences
    conferences[division//division_num]+=teams
    conference_nums.append(division_num)
    
conferences


defaultdict(list,
            {0: ['OnlyFants',
              'Mack Top Greg Bottom',
              'Bend It Like Beckham',
              'The Half Chubbs',
              '#3',
              "Jake's Liver 2",
              'Stafford Infection',
              'White Sox Hat'],
             1: ['Keep Dragon Shoulder',
              'Real Edwards',
              'AutodraftKings',
              '7th Floor Crew',
              'Football Team',
              'Ben’s Koo Young',
              'Bishop Sycamore',
              'Just Win Baby']})

In [97]:
game_types={
    "divisional": 4,#Mandate a divisional game to start and stop the season - hard coded below
    "conference": 4,
    "non_conference": 4
}
game_types_per_week=[]
for k,v in game_types.items():
    game_types_per_week+=[k]*v
random.shuffle(game_types_per_week)
game_types_per_week.insert(0,"divisional")
game_types_per_week+=["divisional"]
game_types_per_week

['divisional',
 'non_conference',
 'non_conference',
 'divisional',
 'non_conference',
 'conference',
 'divisional',
 'conference',
 'divisional',
 'conference',
 'non_conference',
 'divisional',
 'conference',
 'divisional']

# Constraints 
- 1 game max between nondivisional teams. 
- 2 games between divisional

These constraints are relatively simple. Each constraint can be coded by a corresponding universe for each team. To simplify things we will not mix weeks, each week has a shared theme: divisional, non_conference, and conference. When a game is played we will remove that opponent from each teams' universe. 
  
If you wanted to add a "don't play a team 2 weeks in a row rule" you would need to add a short term dont_play set for each team to include all the invalid teams for that team, for that week. 

In [85]:
from itertools import combinations,permutations
import copy


divisional_opponent_universe={}
conference_opponent_universe={}
nonconference_opponent_universe={}

for division, teams in divisions.items():
    for team in teams:
        divisional_opponent_universe[team]=list(set(teams)-set([team]))*2

for conference, teams in conferences.items():
    for team in teams:
        conference_opponent_universe[team]=list(set(teams)-set([team])-set(divisional_opponent_universe[team]))
        #copy is helpful because of pointer vs primitive rules in Python. It may not be necessary in this case though
        nonconference_opponent_universe[team]=copy.deepcopy(conferences[(conference+1)%2])#with 2 conferences this maps 0 to 1 and 1 to 0

universes={
    "divisional": divisional_opponent_universe,
    "conference": conference_opponent_universe,
    "non_conference": nonconference_opponent_universe,
}

universes['divisional']

{'OnlyFants': ['The Half Chubbs',
  'Mack Top Greg Bottom',
  'Bend It Like Beckham',
  'The Half Chubbs',
  'Mack Top Greg Bottom',
  'Bend It Like Beckham'],
 'Mack Top Greg Bottom': ['The Half Chubbs',
  'Bend It Like Beckham',
  'OnlyFants',
  'The Half Chubbs',
  'Bend It Like Beckham',
  'OnlyFants'],
 'Bend It Like Beckham': ['The Half Chubbs',
  'Mack Top Greg Bottom',
  'OnlyFants',
  'The Half Chubbs',
  'Mack Top Greg Bottom',
  'OnlyFants'],
 'The Half Chubbs': ['Mack Top Greg Bottom',
  'OnlyFants',
  'Bend It Like Beckham',
  'Mack Top Greg Bottom',
  'OnlyFants',
  'Bend It Like Beckham'],
 '#3': ["Jake's Liver 2",
  'Stafford Infection',
  'White Sox Hat',
  "Jake's Liver 2",
  'Stafford Infection',
  'White Sox Hat'],
 "Jake's Liver 2": ['#3',
  'Stafford Infection',
  'White Sox Hat',
  '#3',
  'Stafford Infection',
  'White Sox Hat'],
 'Stafford Infection': ['#3',
  "Jake's Liver 2",
  'White Sox Hat',
  '#3',
  "Jake's Liver 2",
  'White Sox Hat'],
 'White Sox Hat':

Notice how divisional has each time in there twice. The other universes are self explanatory

In [86]:

def gen_matchups(all_teams_, universe_):
    '''
    Attempts to generate a week of matchups using each teams universe ()
    '''
    universe_=copy.deepcopy(universe_)
    weekly_teams_playing = set()
    weekly_matchups = []
    teams_remaining=set(all_teams_)
    
    while teams_remaining:
        team_1=teams_remaining.pop()
        candidates=list(set(universe_[team_1])-weekly_teams_playing)
        team_2=random.choice(candidates)
        universe_[team_1].remove(team_2)
        universe_[team_2].remove(team_1)

        weekly_teams_playing=weekly_teams_playing.union(set([team_1,team_2]))
        weekly_matchups.append([team_1,team_2])
        teams_remaining.remove(team_2)
    return weekly_matchups

In [87]:
weekly_teams_playing=defaultdict(set)
games_played={}
weekly_matchups=[]




for week, game_type in enumerate(game_types_per_week):
    #print(week)
        
    print(week,game_type)
    matchups=None
    #Potentially invalid matchups can happen depending on the contstraints so each week will have a given number of attempts. 
    #It's also possible a prior week leads to an impossible state. We will only support rerunning on the week level
    ATTEMPTS=100
    for attempt in range(ATTEMPTS):
        try:
            matchups=gen_matchups(all_teams,universes[game_type])
            break
        except:
            continue
    if not matchups:
        raise 
    else:
        for name, universe in universes.items():
            for team_1,team_2 in matchups:
                for a,b in permutations([team_1,team_2], 2):
                    if b in universe.get(a,[]):                        
                        universes[name][a].remove(b)

        weekly_matchups.append(matchups)

[x[0] for x in weekly_matchups]

0 divisional
1 conference
2 non_conference
3 divisional
4 non_conference
5 non_conference
6 divisional
7 divisional
8 conference
9 divisional
10 non_conference
11 conference
12 conference
13 divisional


[['7th Floor Crew', 'Real Edwards'],
 ['7th Floor Crew', 'Bishop Sycamore'],
 ['7th Floor Crew', 'Bend It Like Beckham'],
 ['7th Floor Crew', 'AutodraftKings'],
 ['7th Floor Crew', "Jake's Liver 2"],
 ['7th Floor Crew', 'The Half Chubbs'],
 ['7th Floor Crew', 'Real Edwards'],
 ['7th Floor Crew', 'Keep Dragon Shoulder'],
 ['7th Floor Crew', 'Just Win Baby'],
 ['7th Floor Crew', 'AutodraftKings'],
 ['7th Floor Crew', 'Mack Top Greg Bottom'],
 ['7th Floor Crew', 'Ben’s Koo Young'],
 ['7th Floor Crew', 'Football Team'],
 ['7th Floor Crew', 'Keep Dragon Shoulder']]

# Validating the Output
We will output the occurrences of each matchup.  
  
No row should have a value greater than 2. And we should see nice c

In [88]:
import numpy as np
N=len(all_teams)
occurences_matrix=np.zeros((N,N))

for weeks_matchups in weekly_matchups:
    
    for team_1,team_2 in weeks_matchups:
        for a,b in permutations([team_1,team_2], 2):
            occurences_matrix[all_teams.index(a)][all_teams.index(b)]+=1
occurences_matrix

array([[0., 2., 2., 2., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1.],
       [2., 0., 2., 2., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1.],
       [2., 2., 0., 2., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0.],
       [2., 2., 2., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1.],
       [1., 1., 1., 1., 0., 2., 2., 2., 1., 0., 1., 0., 1., 0., 0., 1.],
       [1., 1., 1., 1., 2., 0., 2., 2., 1., 0., 0., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 2., 2., 0., 2., 1., 1., 0., 0., 1., 0., 1., 0.],
       [1., 1., 1., 1., 2., 2., 2., 0., 1., 1., 0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 0., 1., 1., 1., 1., 0., 2., 2., 2., 1., 1., 1., 1.],
       [0., 1., 1., 0., 0., 0., 1., 1., 2., 0., 2., 2., 1., 1., 1., 1.],
       [1., 1., 0., 1., 1., 0., 0., 0., 2., 2., 0., 2., 1., 1., 1., 1.],
       [0., 1., 1., 1., 0., 1., 0., 0., 2., 2., 2., 0., 1., 1., 1., 1.],
       [0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 2., 2., 2.],
       [1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1.,

# Input into Yahoo

In [89]:

import pandas as pd

matchups_df=pd.DataFrame(weekly_matchups).T
matchups_df.columns=[x+1 for x in matchups_df.columns]
matchups_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,"[7th Floor Crew, Real Edwards]","[7th Floor Crew, Bishop Sycamore]","[7th Floor Crew, Bend It Like Beckham]","[7th Floor Crew, AutodraftKings]","[7th Floor Crew, Jake's Liver 2]","[7th Floor Crew, The Half Chubbs]","[7th Floor Crew, Real Edwards]","[7th Floor Crew, Keep Dragon Shoulder]","[7th Floor Crew, Just Win Baby]","[7th Floor Crew, AutodraftKings]","[7th Floor Crew, Mack Top Greg Bottom]","[7th Floor Crew, Ben’s Koo Young]","[7th Floor Crew, Football Team]","[7th Floor Crew, Keep Dragon Shoulder]"
1,"[Bishop Sycamore, Football Team]","[Just Win Baby, Real Edwards]","[Bishop Sycamore, White Sox Hat]","[Bishop Sycamore, Ben’s Koo Young]","[Bishop Sycamore, OnlyFants]","[Bishop Sycamore, Stafford Infection]","[Bishop Sycamore, Just Win Baby]","[Bishop Sycamore, Football Team]","[Bishop Sycamore, AutodraftKings]","[Bishop Sycamore, Just Win Baby]","[Bishop Sycamore, Bend It Like Beckham]","[Bishop Sycamore, Keep Dragon Shoulder]","[Bishop Sycamore, Real Edwards]","[Bishop Sycamore, Ben’s Koo Young]"
2,"[Just Win Baby, Ben’s Koo Young]","[Mack Top Greg Bottom, #3]","[Just Win Baby, The Half Chubbs]","[Just Win Baby, Football Team]","[Just Win Baby, #3]","[Just Win Baby, Mack Top Greg Bottom]","[Mack Top Greg Bottom, The Half Chubbs]","[Just Win Baby, Ben’s Koo Young]","[Mack Top Greg Bottom, Jake's Liver 2]","[Mack Top Greg Bottom, OnlyFants]","[Just Win Baby, OnlyFants]","[Just Win Baby, AutodraftKings]","[Just Win Baby, Keep Dragon Shoulder]","[Just Win Baby, Football Team]"
3,"[Mack Top Greg Bottom, Bend It Like Beckham]","[Bend It Like Beckham, White Sox Hat]","[Mack Top Greg Bottom, AutodraftKings]","[Mack Top Greg Bottom, The Half Chubbs]","[Mack Top Greg Bottom, Real Edwards]","[Bend It Like Beckham, Real Edwards]","[Bend It Like Beckham, OnlyFants]","[Mack Top Greg Bottom, OnlyFants]","[Bend It Like Beckham, Stafford Infection]","[Bend It Like Beckham, The Half Chubbs]","[Real Edwards, White Sox Hat]","[Mack Top Greg Bottom, Stafford Infection]","[Mack Top Greg Bottom, White Sox Hat]","[Mack Top Greg Bottom, Bend It Like Beckham]"
4,"[The Half Chubbs, OnlyFants]","[The Half Chubbs, Jake's Liver 2]","[Real Edwards, Stafford Infection]","[Bend It Like Beckham, OnlyFants]","[Bend It Like Beckham, Football Team]","[#3, Football Team]","[#3, Jake's Liver 2]","[Bend It Like Beckham, The Half Chubbs]","[Real Edwards, Ben’s Koo Young]","[Real Edwards, Keep Dragon Shoulder]","[The Half Chubbs, Ben’s Koo Young]","[Bend It Like Beckham, Jake's Liver 2]","[Bend It Like Beckham, #3]","[Real Edwards, AutodraftKings]"
5,"[#3, Stafford Infection]","[AutodraftKings, Football Team]","[#3, Keep Dragon Shoulder]","[Real Edwards, Keep Dragon Shoulder]","[The Half Chubbs, AutodraftKings]","[AutodraftKings, OnlyFants]","[AutodraftKings, Keep Dragon Shoulder]","[Real Edwards, AutodraftKings]","[The Half Chubbs, White Sox Hat]","[#3, White Sox Hat]","[#3, AutodraftKings]","[Real Edwards, Football Team]","[The Half Chubbs, Stafford Infection]","[The Half Chubbs, OnlyFants]"
6,"[AutodraftKings, Keep Dragon Shoulder]","[Ben’s Koo Young, Keep Dragon Shoulder]","[Ben’s Koo Young, OnlyFants]","[#3, Stafford Infection]","[Ben’s Koo Young, White Sox Hat]","[Ben’s Koo Young, Jake's Liver 2]","[Ben’s Koo Young, Football Team]","[#3, Jake's Liver 2]","[#3, OnlyFants]","[Ben’s Koo Young, Football Team]","[Keep Dragon Shoulder, Jake's Liver 2]","[The Half Chubbs, #3]","[AutodraftKings, Ben’s Koo Young]","[#3, White Sox Hat]"
7,"[Jake's Liver 2, White Sox Hat]","[OnlyFants, Stafford Infection]","[Jake's Liver 2, Football Team]","[Jake's Liver 2, White Sox Hat]","[Keep Dragon Shoulder, Stafford Infection]","[Keep Dragon Shoulder, White Sox Hat]","[White Sox Hat, Stafford Infection]","[White Sox Hat, Stafford Infection]","[Keep Dragon Shoulder, Football Team]","[Jake's Liver 2, Stafford Infection]","[Football Team, Stafford Infection]","[White Sox Hat, OnlyFants]","[Jake's Liver 2, OnlyFants]","[Jake's Liver 2, Stafford 

In [90]:
import pandas as pd
pd.DataFrame(weekly_matchups).to_csv("matchups.csv")#save for posterity

In [91]:
divisions

defaultdict(list,
            {0: ['OnlyFants',
              'Mack Top Greg Bottom',
              'Bend It Like Beckham',
              'The Half Chubbs'],
             1: ['#3',
              "Jake's Liver 2",
              'Stafford Infection',
              'White Sox Hat'],
             2: ['Keep Dragon Shoulder',
              'Real Edwards',
              'AutodraftKings',
              '7th Floor Crew'],
             3: ['Football Team',
              'Ben’s Koo Young',
              'Bishop Sycamore',
              'Just Win Baby']})

In [92]:
WEEK_NUM=1
print(game_types_per_week[WEEK_NUM-1])
weekly_matchups[WEEK_NUM-1]

divisional


[['7th Floor Crew', 'Real Edwards'],
 ['Bishop Sycamore', 'Football Team'],
 ['Just Win Baby', 'Ben’s Koo Young'],
 ['Mack Top Greg Bottom', 'Bend It Like Beckham'],
 ['The Half Chubbs', 'OnlyFants'],
 ['#3', 'Stafford Infection'],
 ['AutodraftKings', 'Keep Dragon Shoulder'],
 ["Jake's Liver 2", 'White Sox Hat']]

# Additional Information
Optimization theory and constraint programming deal with scheduling problem. It can be very, very complex. 
- [How the NFL may deal with it](https://www.math.cmu.edu/~af1p/Teaching/OR2/Projects/P56/OR-Final-Paper.pdf)
- [Constraint Programming wikipedia page](https://en.wikipedia.org/wiki/Constraint_programming#:~:text=Constraint%20programming%20(CP)%20is%20a,a%20set%20of%20decision%20variables.)
- [Coursera Course](https://www.coursera.org/learn/discrete-optimization) -- I took this, kinda hard I think I flunked by the end but did learn some useful stuff.